In [22]:
import os

In [36]:
session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0             |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |sales         |low       |
|1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                      

In [38]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [39]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [41]:
import pandas as pd
# df = pd.read_csv("HR_CHURN.csv")
# df = session.table("HR_CHURN")
# snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

In [42]:
input_data_frame.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |sales         |medium    |
|2             |0.11                  |0.88               |7                 |272                     |4                     |0                |1       |0                      

In [43]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|2.0       |7.0           |2             |0.11                  |0.88               |7                 |272                     |4                     |0                |1     

In [44]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

/tmp/pip_packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DummyClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/pip_packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/pip_packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GradientBoostingClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For mor

In [45]:
pred_snow_df = pipeline.predict(apply_label_encoding(test_df))

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [16]:
pred_snow_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"OUTPUT_SALARY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |3             |0.72                  |0.87               |5                 |223                     |5                     |0                |1       |0                        |2.0              |
|1.0       |7.0           |6             |0.1                   |0.77               |6                 |

In [17]:
pred_snow_df.to_pandas()

,SALARY,DEPARTMENT,Unnamed: 0,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,OUTPUT_SALARY
0,1.0,2.0,28,0.41,0.46,2,128,3,0,1,0,1.0
1,2.0,0.0,62,0.10,0.95,6,244,5,0,1,0,2.0
2,1.0,7.0,94,0.09,0.83,6,255,4,0,1,0,1.0
3,1.0,9.0,122,0.75,1.00,4,216,6,0,1,0,1.0
4,1.0,2.0,155,0.11,0.80,6,304,4,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3697,1.0,8.0,14869,0.38,0.53,2,146,3,0,1,0,1.0
3698,2.0,2.0,14897,0.11,0.92,6,269,4,0,1,0,2.0
3699,1.0,8.0,14922,0.11,0.86,6,286,4,0,1,0,1.0
3700,1.0,7.0,14958,0.42,0.52,2,134,3,0,1,0,1.0


In [46]:
from snowflake.ml.registry import Registry

In [ ]:
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": "Enlightme#2024",
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC"
}

In [47]:
model_registry = Registry(session=session, 
                                       database_name="FIRST_DB",
                                       schema_name="PUBLIC"
                                       )

In [49]:
import platform
platform.python_version()

'3.10.13'

In [50]:
model_registry.log_model(
                   pipeline,
                   model_name="TestModel",
                   version_name="v1",
                   comment="test model",
                   conda_dependencies=["scikit-learn==1.3.2"],
                   metrics={},
                   sample_input_data=input_data_frame,
                   python_version="3.10.13"  
                  )

/opt/conda/lib/python3.10/contextlib.py:135: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/tmp/pip_packages/snowflake/ml/model/_packager/model_packager.py:92: UserWarning: Inferring model signature from sample input or providing model signature for Snowpark ML Modeling model is not required. Model signature will automatically be inferred during fitting. 
  handler.save_model(


In [54]:
model_obj = model_registry.get_model(model_name="TestModel")

In [55]:
model = model_obj.version(version_name="v1")

In [58]:
inptu_df ,test_df = session.table("HR_CHURN").randomSplit([0.75, 0.25])

In [59]:
test_df = apply_label_encoding(test_df)

In [75]:
test_df_pandas = test_df.to_pandas()

In [76]:
session.write_pandas(test_df_pandas,table_name="TEST_DATA_HR",overwrite=True).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [80]:
test_df_fetch = session.table("TEST_DATA_HR")

In [81]:
test_df_fetch.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [82]:
model.run(test_df_fetch, function_name="predict").to_pandas()

,SALARY,DEPARTMENT,Unnamed: 0,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,OUTPUT_SALARY
0,1.0,3.0,34,0.84,0.87,4,246,6,0,1,0,1.0
1,2.0,6.0,66,0.43,0.54,2,153,3,0,1,0,2.0
2,1.0,7.0,94,0.09,0.83,6,255,4,0,1,0,1.0
3,1.0,8.0,130,0.10,0.83,6,292,4,0,1,0,1.0
4,1.0,7.0,163,0.79,0.89,5,239,5,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7447,2.0,5.0,14821,0.09,0.95,7,256,4,0,1,0,2.0
7448,2.0,9.0,14859,0.45,0.54,2,129,3,0,1,0,2.0
7449,1.0,7.0,14888,0.39,0.49,2,142,3,0,1,0,1.0
7450,2.0,7.0,14945,0.14,0.75,4,277,5,1,1,0,1.0


In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)